In [ ]:
!pip install -q transformers datasets
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.0 MB/s eta 0:00:00


In [ ]:
import numpy as np

In [ ]:
dataset = load_dataset("LucasThil/miniwobplusplus_episodes")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['episodes', 'actions'],
        num_rows: 16794
    })
})

In [ ]:
# Actions we keep: either click or type
action_labels = ['click', 'keydown']

# Reformat the full action_history into a list and remove unecessary actions
def clean_action_history(action_history):
  actions = action_history.split('}{')
  clean_actions = []  # performed actions
  refs = []           # ref number

  for a in actions:
    action_ref = a.replace('{', "").replace('}', '').split(', ')
    action = action_ref[0].split('action: ')[1]
    ref = action_ref[1].split('ref: ')[1]
    if action in action_labels:
      clean_actions.append(action)
      refs.append(ref)

  return clean_actions, refs

# Fields to drop in the html:
drop_fields = ['children', 'bgColor', 'width', 'height', 'focused', 'fgColor', 'right', 'top', 'left', 'bottom', 'class', 'classes']

# Clean the html file
def clean_html(html_elements):
  html = ""

  starting_tag = html_elements[0]
  # Reput the opening tag
  if '<' not in starting_tag:
    starting_tag = '<' + starting_tag
  if '>' not in starting_tag:
    starting_tag += '>'
  html += starting_tag

  for element in html_elements[1:]:

    # Get the tag element
    starting_tag = element.split(' ')[0]
    # Reput the opening tag
    if '<' not in starting_tag:
      html += '<' + starting_tag

    # Get the interior fields, but discard the first being the tag
    parts = element.split('" ')[1:]

    for p in parts:
      field = p.split('=')[0]
      if field not in drop_fields:
        html += " " + p

    # Reput closing tag
    if '>' not in element:
      html += '>'

  return html



# Clean the episode entries by removing unecessary actions and cleaning the html
def clean_episode(row):
  episodes_str = row['episodes'].split("\', \'")

  # We can just get the last episode as it contains the full action history
  episode = episodes_str[len(episodes_str)-1]

  # Split to get action history, Uterance, and HTML
  episode_parts = episode.split('} ')
  flag = False # in case we get a special case of utterance
  if len(episode_parts) != 2:
    #print(f'problem splitting action history {len(episode_parts)} {episode}')
    flag = True
    #print(f'episode_parts: {episode_parts}')
    #return

  action_history = episode_parts[0] + "}"

  actions, refs = clean_action_history(action_history)

  if not flag:
    uterance_html = episode_parts[1].lower().split(' <body')
    uterance = uterance_html[0]
    html = '<body' + uterance_html[1]
  else:
    uterance = episode_parts[1].lower()
    #print(f'uterance_html: {uterance}')
    html = episode_parts[2]
    #print(f'html: {html}')

  html_elements = html.replace(']', '').split("><")
  html = clean_html(html_elements)

  return html, uterance, actions, refs



# Short test

In [ ]:
example = dataset['train'][1001]
example

{'episodes': '[\'Enter "7" into the text field and press Submit. <BODY ref="7"><DIV id="wrap" ref="8"><DIV id="area" ref="9"><DIV id="form" ref="10"><INPUT_text id="tt" ref="11"></INPUT_text><BUTTON id="subbtn" classes="secondary-action" ref="12" text="Submit"></BUTTON></DIV></DIV></DIV></BODY>\', \'{action: mousedown, ref: 7} Enter "7" into the text field and press Submit. <BODY ref="7"><DIV id="wrap" ref="8"><DIV id="area" ref="9"><DIV id="form" ref="10"><INPUT_text id="tt" ref="11" recordingTarget="True"></INPUT_text><BUTTON id="subbtn" classes="secondary-action" ref="12" text="Submit"></BUTTON></DIV></DIV></DIV></BODY>\', \'{action: mousedown, ref: 7}{action: mousedown, ref: 7} Enter "7" into the text field and press Submit. <BODY ref="7"><DIV id="wrap" ref="8"><DIV id="area" ref="9"><DIV id="form" ref="10"><INPUT_text id="tt" ref="11" recordingTarget="True"></INPUT_text><BUTTON id="subbtn" classes="secondary-action" ref="12" text="Submit"></BUTTON></DIV></DIV></DIV></BODY>\', \'{a

In [ ]:
# 2
# We want to iterate through the episodes, and gather the action history.
# For each action step, we ensure that it is not a duplicate of the previous one.
# If there are duplicates, we'll only keep the last one as we assume it was the rightly performed action.
# Example for typing, there are multiple 'keydown' events, but an agent would paste the text directly and thus only one action would be needed.
# We discard any click,1 which corresponds to a click to the <body> element
#
# For each keydown action, we only keep the last one for a targetted element.
# We obtain the correct string to put by checking the 'value' field of the targeted element in the last episode.
#
# Clean the episode entries by removing unecessary actions and cleaning the html
# clean_episode2 is that updated version
def clean_episode2(row):
  episodes_str = row['episodes'].replace('[', "").replace("]", "").replace("\', \'", "\', \'Ø").replace("\'", "").split("Ø")

  # Uterance of the experiment
  instruction = episodes_str[0].split('<')[0]

  # Flag if the input instruction contains a JSON instruction. We can only check the first episode of the experiment as it has no action history
  # We do that to tell the loop below to discard the last '{}' segment when parsing the action history
  # TODO: maybe no need, just get the first value
  if "{" in instruction:
    json_instruction = True
  else:
    json_instruction = False

  # We take the full html miniwob snippet as they are quite small in the MiniWob benchmark
  html_snippets = []

  # Actions types
  actions = []

  # Ref elements
  refs = []

  # Text to input for each 'keydown' action
  keydown_texts = []

  i = 0
  past_action = ""
  # The counter below keeps track of repeated actions to understand which html snippet to use.
  # If the action happens to be repeated, we update the last html_snippet appended with the new found one
  for episode in episodes_str:
    if i != 0:
      # Get last action, should be the left-most element of the parsed actions
      last_action = episode.split('}')[0]
      # Check we get the right kind of action
      if ('click' in last_action or 'keydown' in last_action) and 'dblclick' not in last_action:
        if last_action != '{action: click, ref: 1' and last_action != '{action: keydown, ref: 1' and last_action != past_action:
          #print(f'last_action {last_action}')
          past_action = last_action

          action_ref = last_action.replace('{', "").split(', ')
          action_episode = action_ref[0].split('action: ')[1]
          ref_episode = action_ref[1].split('ref: ')[1]
          actions.append(action_episode)
          refs.append(ref_episode)

          # Append html snippet
          html_snip = episode.lower().replace('<body', 'Ø<body').replace(' recordingtarget="true"', "").replace('  ', ' ').split('Ø')[1]
          html_snippets.append(html_snip.lower())
        else:
          # Found a duplicated action, so we update the lastest snippet with the current one as it is more recent.
          # ==> DISCARDED: we only update it when the action is different, otherwise we have the resulting HTML instead of the initial one.
          #html_snip = episode.replace('<BODY', 'Ø<BODY').split('Ø')[1]
          #html_snippets[len(html_snippets)-1] = html_snip
          continue

    i += 1

  # Loop over all actions and find corresponding keydown input text
  # Get the refs of the elements targeted by a keydown action and find in the final html what was their text input
  # Get the elements of the last episode:
  elements = episodes_str[len(episodes_str)-1].lower().replace('<body', 'Ø<body').replace('  ', ' ').split('Ø')[1].split('>')
  j = 0
  while j < len(actions):
    # When action is keydown, find the corresponding value element with that ref
    if actions[j] == 'keydown' :
      #print(f'current_action_keydown: {actions[j]}')
      for el in elements:
        # Find the element with the correct ref
        if 'ref="' + refs[j] in el:
          # Get the input value we're looking for
          #print(f'found element: {el}')
          input_text = el.split('value="')[1].split('"')[0]
          #print(f'input_text: {input_text}')
          keydown_texts.append(input_text)
          break
    else:
      keydown_texts.append('')
    j += 1

  # Now create the correct snippet episodes by iterating and appending the action_history with its correct html snippets
  i = 0
  history_episodes = []
  history_actions = ""
  while i < len(html_snippets):
    if i == 0:
      history_episodes.append(instruction + ' ' + html_snippets[i])
      history_actions = "{" + actions[i] + "," + str(refs[i]) + "," + keydown_texts[i] + "}"
    else:
      history_episodes.append(history_actions + ' ' + instruction + ' ' + html_snippets[i])
      history_actions += "{" + actions[i] + "," + str(refs[i]) + "," + keydown_texts[i] + "}"
    i += 1

  return history_episodes, instruction, html_snippets, actions, refs, keydown_texts


In [ ]:
example = dataset['train'][22]
history_episodes, instruction, html_snippets, actions, refs, keydown_texts = clean_episode2(example)

#history_episodes, instruction, html_snippets, actions, refs, keydown_texts
len(history_episodes), len(instruction), len(html_snippets), len(actions), len(refs), len(keydown_texts)

(2, 43, 2, 2, 2, 2)

In [ ]:
history_episodes

['Expand the section below and click submit.  <body ref="5"><div id="wrap" ref="6"><div id="area" classes="ui-accordion ui-widget ui-helper-reset" ref="7"><h3 id="ui-id-10" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons" ref="1"><t ref="-6" text="section #13"></t></h3><p id="ui-id-12" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons" ref="2"><button id="subbtn" classes="secondary-action" ref="3" text="submit"></button></p></div></div></body>, ',
 '{click,5,} Expand the section below and click submit.  <body ref="5"><div id="wrap" ref="6"><div id="area" classes="ui-accordion ui-widget ui-helper-reset" ref="7"><h3 id="ui-id-10" classes="ui-accordion-header ui-corner-top ui-state-default ui-accordion-icons ui-accordion-header-active ui-state-active" ref="1"><t ref="-18" text="section #13"></t></h3><div id="ui-id-11" classes="ui-accordion-con

In [ ]:
html_snippets

['<body ref="5"><div id="wrap" ref="6"><div id="area" classes="ui-accordion ui-widget ui-helper-reset" ref="7"><h3 id="ui-id-10" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons" ref="1"><t ref="-6" text="section #13"></t></h3><p id="ui-id-12" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons" ref="2"><button id="subbtn" classes="secondary-action" ref="3" text="submit"></button></p></div></div></body>, ',
 '<body ref="5"><div id="wrap" ref="6"><div id="area" classes="ui-accordion ui-widget ui-helper-reset" ref="7"><h3 id="ui-id-10" classes="ui-accordion-header ui-corner-top ui-state-default ui-accordion-icons ui-accordion-header-active ui-state-active" ref="1"><t ref="-18" text="section #13"></t></h3><div id="ui-id-11" classes="ui-accordion-content ui-corner-bottom ui-helper-reset ui-widget-content ui-accordion-content-active" ref="4" text="

In [ ]:
print(instruction)

Expand the section below and click submit. 


In [ ]:
print(actions)

['click', 'click']


# checkpoint for CC-Net prep, jump to corresponding section

In [ ]:
import traceback

# Loop do for the entire dataset
history_episodes = []
instruction = []
html_snippets = []
actions = []
refs = []
keydown_texts = []

i = 0
error_rows = [] # Errors when trying to clean_episode2
error_empty = []  # Errors when retrieving actions lengths being 0

for row in dataset['train']:
  if i % 100 == 0:
    print(i)
  try:
    history_episodes_r, instruction_r, html_snippets_r, actions_r, refs_r, keydown_texts_r = clean_episode2(row)
  except Exception as e:
    error_rows.append(row)
    i += 1
    continue
  if len(actions_r) > 0:
    if len(history_episodes_r) == len(html_snippets_r) and len(html_snippets_r) == len(actions_r) and len(refs_r) == len(keydown_texts_r):
      j = 0
      while j < len(history_episodes_r):
        history_episodes.append(history_episodes_r[j])
        instruction.append(instruction_r)
        html_snippets.append(html_snippets_r[j])
        actions.append(actions_r[j])
        refs.append(refs_r[j])
        keydown_texts.append(keydown_texts_r[j])
        j += 1
    else:
      print(f"Error processing, not equal lengths at index {i}: {row}")
      break
  else:
    #print(f'No action length found: {actions_r}, row: {row}')
    error_empty.append(row)
  i += 1

In [ ]:
print(f'found total of {len(error_rows)} error rows that were discarded')
print(f'found total of {len(error_empty)} that had no actions')
len(history_episodes), len(instruction), len(html_snippets), len(actions), len(refs), len(keydown_texts)

found total of 471 error rows that were discarded
found total of 1829 that had no actions


(75039, 75039, 75039, 75039, 75039, 75039)

In [ ]:
print(history_episodes[96])
print(history_episodes[97])
print(history_episodes[98])
print(history_episodes[99])

{click,15,} {"Departure City":"Charlotte","Destination City":"Portland","Ticket Type":"One-way","Departure Day":14,"Passengers":3,"Seat type":"Coach"}  <body ref="1"><a classes="banner" ref="2"><img classes="alaska_logo" ref="3"></img></a><div id="main" ref="4"><div id="starth1" classes="header" ref="5"><h1 text="book a flight" classes="h1" ref="6"></h1></div><form id="searchform" classes="miniwob-main-form" ref="7"><div classes="form-row group" ref="8"><div classes="left" ref="9"><label text="one-way" classes="css-lbl" ref="10"></label></div><div id="use-miles-div" classes="right" ref="11"><label text="use miles" classes="css-lbl" ref="12"></label></div></div><div id="geo-from-wrap" classes="form-row geo-wrap" ref="13"><label classes="text-label" ref="14"><t text="from" ref="-33"></t><input_text id="geo-from" classes="text-input-pad" ref="15"></input_text></label><div classes="cleartxt" ref="16"></div><div id="geo-from-button" classes="geo-button" ref="17"><span text="geolocation" cla

In [ ]:
print(history_episodes[1002])
print(actions[1002])
print(refs[1002])

Select all the shades of blue and press Submit.  <body ref="17"><div id="wrap" ref="18"><div id="area" ref="19"><span ref="1"></span><span ref="2"></span><span ref="3"></span><span ref="4"></span><span ref="5"></span><span ref="6"></span><span ref="7"></span><span ref="8"></span><span ref="9"></span><span ref="10"></span><span ref="11"></span><span ref="12"></span><button id="submit" classes="secondary-action" ref="13" text="submit"></button></div></div></body>, 
click
17


Looks good, saving as csv and then dataset

In [ ]:
df = {
    "history_episodes": history_episodes,
    "instruction": instruction,
    "html_snippets": html_snippets,
    "actions": actions,
    "refs": refs,
    "keydown_texts": keydown_texts,
}

import pandas as pd
df = pd.DataFrame(df)
df.to_csv("/content/sample_data/train.csv", index=False)

KeyboardInterrupt: ignored

In [ ]:
from datasets import load_dataset
file_dict = {
  "train" : "/content/sample_data/train.csv",
}
df_train = load_dataset("csv", data_files=file_dict)

In [ ]:
#df_train.push_to_hub('LucasThil/miniwob_plusplus_T5_unbounded')

In [ ]:
import matplotlib.pyplot as plt

plt.hist(actions, bins=len(set(actions)))
# Add axis labels and title
plt.xlabel('Event Type')
plt.ylabel('Count')
plt.title('Histogram of Click and Keydown Events')

# Show plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.hist(refs, bins=len(set(refs)))
# Add axis labels and title
plt.xlabel('Ref Number')
plt.ylabel('Count')
plt.title('Histogram of Refs Numbers')

# Show plot
plt.show()

# Now we need to replace all ref by randomizing its values with dictionaries

In [ ]:
example = dataset['train'][6000]
history_episodes, instruction, html_snippets, actions, refs, keydown_texts = clean_episode2(example)
history_episodes, instruction, html_snippets, actions, refs, keydown_texts

In [ ]:
import re
import random

# General notes:
#   In total, we'll perform two loops: one for creating initial snippets, the other to randomize the ref numbers, so we are flexible in re-using the code for processing the dataset further

# For each row, html_snippet:
# 1 Get all refs from the (action, ref) tuples
# 2 Get all refs from the html_snippet (we'll update them with the .replace so we are correct with the values)
# 3 Create a random number generator in the range of html_refs without replacement
# 4 Map refs from the (action, ref) tuples to their new number
# 5 Remove all rows where the target ref is the body element
# 6 Replace these refs to the html_snippet
# 7 Create Episodes with Action History

def randomize_refs(history_episodes, instruction, html_snippets, actions, refs, keydown_texts):

  # 2 - We'll get the refs from all the html episodes. We then concatenate them and get the unique values.
  ref_pattern = r'ref="(\d+)"'

  k = 0
  # Store all ref found in the episodes
  raw_refs = []
  while k < len(html_snippets):
    ref_numbers_episode = [int(m.group(1)) for m in re.finditer(ref_pattern, html_snippets[k])]
    raw_refs += ref_numbers_episode
    k += 1

  # Now get the unique references found
  raw_refs = list(set(raw_refs))
  # print the list of integers

  # 3 - Create the random number generator in that range without replacement
  # generate a list of all numbers in the range
  all_numbers = list(range(1, len(raw_refs)+1))

  # select a random sample of unique numbers from the list
  # Create the random refs
  random_refs = random.sample(all_numbers, len(raw_refs))

  # 4 - Create the dictionary to perform the mapping between the two
  raw2rand = dict(zip(raw_refs, random_refs))
  #rand2raw = dict(zip(random_refs, raw_refs))

  # 5 - Get the ref of the body element
  body_ref = int(html_snippets[len(html_snippets)-1].split('<body ref="')[1].split('"')[0])

  # 6 - Loop to update all ref numbers in the html snippets, and discard the episodes where the target action is targeting the body element
  #     If we find an action targeting the body element, ensure that it is not present in the action_history
  k = 0

  # New lists to contain the updated values
  new_html_snippets = []
  new_actions = []
  new_refs = []
  new_keydown_texts = []

  while k < len(html_snippets):
    if int(refs[k]) == body_ref:
      # Found an action that targets the body ref, discard it and continue
      k += 1
      continue
    else:
      # Update all numbers in the html_snippet + put lowercase
      updating_html = html_snippets[k]
      for new_ref in raw2rand:
        # We add the 'Ø' symbol to mark references already replaced
        updating_html = updating_html.replace('ref="' + str(new_ref) + '"', 'ref="' + str(raw2rand[new_ref]) + 'Ø"')
      # Append the transformerd html snippet to the new_list, and remove the 'Ø' marking the updated values
      new_html_snippets.append(updating_html.replace('Ø', ''))
      # Do the same for the corresponding values
      new_actions.append(actions[k])
      new_refs.append(raw2rand[int(refs[k])])
      new_keydown_texts.append(keydown_texts[k])
      k += 1

  # Now create the correct snippet episodes by iterating and appending the action_history with its correct html snippets
  i = 0
  new_history_episodes = []
  history_actions = ""
  while i < len(new_html_snippets):
    if i == 0:
      new_history_episodes.append(instruction + ' ' + new_html_snippets[i])
      history_actions = "{" + new_actions[i] + "," + str(new_refs[i]) + "," + new_keydown_texts[i] + "}"
    else:
      new_history_episodes.append(history_actions + ' ' + instruction + ' ' + new_html_snippets[i])
      history_actions += "{" + new_actions[i] + "," + str(new_refs[i]) + "," + new_keydown_texts[i] + "}"
    i += 1

  return new_history_episodes, new_html_snippets, instruction, new_actions, new_refs, new_keydown_texts

history_episodes, new_html_snippets, new_instruction, new_actions, new_refs, new_keydown_texts = randomize_refs(history_episodes, instruction, html_snippets, actions, refs, keydown_texts)
history_episodes, new_html_snippets, new_instruction, new_actions, new_refs, new_keydown_texts

(['Expand the section below and click submit.  <body ref="6"><div id="wrap" ref="4"><div id="area" classes="ui-accordion ui-widget ui-helper-reset" ref="3"><h3 id="ui-id-1" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons ui-state-hover ui-state-focus" ref="7"><t ref="-6" text="section #9"></t></h3><p id="ui-id-3" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons" ref="2"><button id="subbtn" classes="secondary-action" ref="5" text="submit"></button></p></div></div></body>, ',
  '{click,7,} Expand the section below and click submit.  <body ref="6"><div id="wrap" ref="4"><div id="area" classes="ui-accordion ui-widget ui-helper-reset" ref="3"><h3 id="ui-id-1" classes="ui-accordion-header ui-corner-top ui-state-default ui-accordion-icons ui-accordion-header-active ui-state-active" ref="7"><t ref="-12" text="section #9"></t></h3><div id="ui-id-2"

In [ ]:
instruction

'Select members and click Submit. '

In [ ]:
rd_history_episodes = []
rd_html_snippets = []
rd_instruction = []
rd_actions = []
rd_refs = []
rd_keydown_texts = []

# Keep track of some metrics
failed_lengths = []
empty_clean = []
failed_random = []

# Now loop for all
i = 0
for row in dataset['train']:

  if i % 100 == 0:
    print(i)
  try:
    history_episodes, instruction, html_snippets, actions, refs, keydown_texts = clean_episode2(row)
  except Exception as e:
      error_rows.append(row)
      i += 1
      continue
  if len(actions) > 0:
    if len(history_episodes) == len(html_snippets) and len(html_snippets) == len(actions) and len(refs) == len(keydown_texts):
      new_history_episodes, new_html_snippets, new_instruction, new_actions, new_refs, new_keydown_texts = randomize_refs(history_episodes, instruction, html_snippets, actions, refs, keydown_texts)

      if len(new_history_episodes) > 0 and len(new_history_episodes) == len(new_html_snippets) and len(new_html_snippets) == len(new_keydown_texts) and len(new_actions) == len(new_refs):

        # Append for all
        j = 0
        while j < len(new_history_episodes):
          rd_history_episodes.append(new_history_episodes[j])
          rd_html_snippets.append(new_html_snippets[j])
          rd_instruction.append(new_instruction)
          rd_actions.append(new_actions[j])
          rd_refs.append(new_refs[j])
          rd_keydown_texts.append(new_keydown_texts[j])
          j += 1
      else:
        failed_random.append(row)
    else:
      failed_lengths.append(row)
  else:
    empty_clean.append(row)

  i += 1

print(f'failed_lengths {len(failed_lengths)} empty_clean = {len(empty_clean)} failed_random = {len(failed_random)}')
len(rd_history_episodes), len(rd_html_snippets), len(rd_instruction), len(rd_actions), len(rd_refs), len(rd_keydown_texts)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
failed_lengths 0 empty_clean = 1829 failed_random = 1929


(60321, 60321, 60321, 60321, 60321, 60321)

In [ ]:
print(f'failed_lengths {len(failed_lengths)} empty_clean = {len(empty_clean)} failed_random = {len(failed_random)}')

failed_lengths 0 empty_clean = 1829 failed_random = 1929


In [ ]:
df = {
    "history_episodes": rd_history_episodes,
    "instruction": rd_instruction,
    "html_snippets": rd_html_snippets,
    "actions": rd_actions,
    "refs": rd_refs,
    "keydown_texts": rd_keydown_texts,
}

import pandas as pd
df = pd.DataFrame(df)
df.to_csv("/content/sample_data/train.csv", index=False)

In [ ]:
from datasets import load_dataset
file_dict = {
  "train" : "/content/sample_data/train.csv",
}
df_train = load_dataset("csv", data_files=file_dict)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e80341f7e7db04b8/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df.head(15)

,history_episodes,instruction,html_snippets,actions,refs,keydown_texts
0,Expand the section below and click submit. <b...,Expand the section below and click submit.,"<body ref=""3""><div id=""wrap"" ref=""6""><div id=""...",click,7,
1,"{click,7,} Expand the section below and click ...",Expand the section below and click submit.,"<body ref=""3""><div id=""wrap"" ref=""6""><div id=""...",click,4,
2,Expand the section below and click submit. <b...,Expand the section below and click submit.,"<body ref=""3""><div id=""wrap"" ref=""1""><div id=""...",click,4,
3,"{click,4,} Expand the section below and click ...",Expand the section below and click submit.,"<body ref=""3""><div id=""wrap"" ref=""1""><div id=""...",click,6,
4,Expand the section below and click submit. <b...,Expand the section below and click submit.,"<body ref=""4""><div id=""wrap"" ref=""3""><div id=""...",click,2,
5,"{click,2,} Expand the section below and click ...",Expand the section below and click submit.,"<body ref=""4""><div id=""wrap"" ref=""3""><div id=""...",click,6,
6,Expand the section below and click submit. <b...,Expand the section below and click submit.,"<body ref=""3""><div id=""wrap"" ref=""2""><div id=""...",click,1,
7,Expand the section below and click submit. <b...,Expand the section below and click submit.,"<body ref=""6""><div id=""wrap"" ref=""2""><div id=""...",click,1,
8,Expand the section below and click submit. <b...,Expand the section below and click submit.,"<body ref=""1""><div id=""wrap"" ref=""3""><div id=""...",click,6,
9,Expand the section below and click submit. <b...,Expand the section below and click submit.,"<body ref=""3""><div id=""wrap"" ref=""2""><div id=""...",click,1,


In [ ]:
df_train

DatasetDict({
    train: Dataset({
        features: ['history_episodes', 'instruction', 'html_snippets', 'actions', 'refs', 'keydown_texts'],
        num_rows: 60321
    })
})

In [ ]:
#df_train.push_to_hub('LucasThil/miniwob_plusplus_T5_randomized_ref2')

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(df_train["train"])

,history_episodes,instruction,html_snippets,actions,refs,keydown_texts
0,"Select words similar to televisions and click Submit. <body ref=""13""><div id=""wrap"" ref=""6""><div id=""area"" ref=""1""><div id=""boxes"" ref=""7""><label ref=""10""><input type=""checkbox"" id=""ch0"" ref=""12"" value=""true""></input><t ref=""-26"" text=""tv""></t></label><label ref=""5""><input type=""checkbox"" id=""ch1"" ref=""9"" value=""false""></input><t ref=""-27"" text=""halt""></t></label><label ref=""2""><input type=""checkbox"" id=""ch2"" ref=""8"" value=""false""></input><t ref=""-28"" text=""assassinate""></t></label><label ref=""4""><input type=""checkbox"" id=""ch3"" ref=""11"" value=""false""></input><t ref=""-29"" text=""sad""></t></label><label ref=""13""><input type=""checkbox"" id=""ch4"" ref=""6"" value=""false""></input><t ref=""-30"" text=""sinful""></t></label></div><button id=""subbtn"" classes=""secondary-action"" ref=""3"" text=""submit""></button></div></div></body>,",Select words similar to televisions and click Submit.,"<body ref=""13""><div id=""wrap"" ref=""6""><div id=""area"" ref=""1""><div id=""boxes"" ref=""7""><label ref=""10""><input type=""checkbox"" id=""ch0"" ref=""12"" value=""true""></input><t ref=""-26"" text=""tv""></t></label><label ref=""5""><input type=""checkbox"" id=""ch1"" ref=""9"" value=""false""></input><t ref=""-27"" text=""halt""></t></label><label ref=""2""><input type=""checkbox"" id=""ch2"" ref=""8"" value=""false""></input><t ref=""-28"" text=""assassinate""></t></label><label ref=""4""><input type=""checkbox"" id=""ch3"" ref=""11"" value=""false""></input><t ref=""-29"" text=""sad""></t></label><label ref=""13""><input type=""checkbox"" id=""ch4"" ref=""6"" value=""false""></input><t ref=""-30"" text=""sinful""></t></label></div><button id=""subbtn"" classes=""secondary-action"" ref=""3"" text=""submit""></button></div></div></body>,",click,12,None
1,"{click,20,}{click,12,}{click,7,} Select 12rh2X4, Hjfr, iMDDWUP, Jc0Wr4A, Rg and click Submit. <body ref=""8""><div id=""wrap"" ref=""1""><div id=""area"" ref=""9""><div id=""boxes-left"" ref=""11""><label ref=""3""><input type=""checkbox"" id=""ch0"" ref=""20"" value=""true""></input><t ref=""-162"" text=""hjfr""></t></label><label ref=""4""><input type=""checkbox"" id=""ch1"" ref=""12"" value=""true""></input><t ref=""-163"" text=""imddwup""></t></label><label ref=""19""><input type=""checkbox"" id=""ch2"" ref=""7"" value=""true""></input><t ref=""-164"" text=""jj2""></t></label><label ref=""16""><input type=""checkbox"" id=""ch3"" ref=""13"" value=""true""></input><t ref=""-165"" text=""jc0wr4a""></t></label></div><div id=""boxes-right"" ref=""14""><label ref=""10""><input type=""checkbox"" id=""ch4"" ref=""6"" value=""false""></input><t ref=""-166"" text=""rg""></t></label><label ref=""5""><input type=""checkbox"" id=""ch5"" ref=""17"" value=""false""></input><t ref=""-167"" text=""12rh2x4""></t></label><label ref=""18""><input type=""checkbox"" id=""ch6"" ref=""2"" value=""false""></input><t ref=""-168"" text=""4znqt""></t></label></div><button id=""subbtn"" classes=""secondary-action"" ref=""15"" text=""submit""></button></div></div></body>,","Select 12rh2X4, Hjfr, iMDDWUP, Jc0Wr4A, Rg and click Submit.","<body ref=""8""><div id=""wrap"" ref=""1""><div id=""area"" ref=""9""><div id=""boxes-left"" ref=""11""><label ref=""3""><input type=""checkbox"" id=""ch0"" ref=""20"" value=""true""></input><t ref=""-162"" text=""hjfr""></t></label><label ref=""4""><input type=""checkbox"" id=""ch1"" ref=""12"" value=""true""></input><t ref=""-163"" text=""imddwup""></t></label><label ref=""19""><input type=""checkbox"" id=""ch2"" ref=""7"" value=""true""></input><t ref=""-164"" text=""jj2""></t></label><label ref=""16""><input type=""checkbox"" id=""ch3"" ref=""13"" value=""true""></input><t ref=""-165"" text=""jc0wr4a""></t></label></div><div id=""boxes-right"" ref=""14""><label ref=""10""><input type=""checkbox"" id=""ch4"" ref=""6"" value=""false

In [ ]:
df_train["train"][30000] #HH

{'history_episodes': '{click,1,}{click,4,} Select w09kRR, oQiCL, hbM6wGo, 8XKy, hf, 5YuGE, DW5Wc and click Submit.  <body ref="24"><div id="wrap" ref="2"><div id="area" ref="19"><div id="boxes-left" ref="11"><label ref="16"><input type="checkbox" id="ch0" ref="5" value="false"></input><t ref="-154" text="8xky"></t></label><label ref="14"><input type="checkbox" id="ch1" ref="10" value="true"></input><t ref="-155" text="hbm6wgo"></t></label><label ref="22"><input type="checkbox" id="ch2" ref="1" value="true"></input><t ref="-156" text="w09krr"></t></label><label ref="13"><input type="checkbox" id="ch3" ref="4" value="true"></input><t ref="-157" text="oqicl"></t></label><label ref="17"><input type="checkbox" id="ch4" ref="8" value="false"></input><t ref="-158" text="egk69p"></t></label></div><div id="boxes-right" ref="15"><label ref="21"><input type="checkbox" id="ch5" ref="7" value="false"></input><t ref="-159" text="hf"></t></label><label ref="3"><input type="checkbox" id="ch6" ref="18"

# Explore Dataset

In [1]:
dataset = load_dataset("LucasThil/miniwob_plusplus_T5_randomized_ref2")

NameError: ignored

In [ ]:
dataset

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

# Hierarchical Dataset

We'll use one of the previous computed dataset with random refs and process it by breaking down the instruction in different parts, and linked them with the actions or episodes presented

In [ ]:
dataset

In [ ]:
dataset['train'][6969]

{'episodes': '[\'pull up Leanna\\\'s last email and write back "Fringilla duis." <BODY ref="127"><DIV id="wrap" ref="128"><DIV id="area" ref="129"><DIV id="main" ref="1"><DIV id="main-header" ref="2"><H2 ref="3" text="Primary"></H2><SPAN id="open-search" ref="4"></SPAN></DIV><DIV classes="email-thread" ref="5"><DIV classes="email-left" ref="6"><DIV classes="email-sender" ref="7" text="Lilla"></DIV><DIV classes="email-subject" ref="8" text="In."></DIV><DIV classes="email-body" ref="9" text="Nulla proin sus.."></DIV></DIV><DIV classes="email-right" ref="10"><DIV classes="email-actions" ref="11"><SPAN classes="trash" ref="12"></SPAN><SPAN classes="star" ref="13"></SPAN></DIV></DIV></DIV><DIV classes="email-thread" ref="14"><DIV classes="email-left" ref="15"><DIV classes="email-sender" ref="16" text="Brigitte"></DIV><DIV classes="email-subject" ref="17" text="Leo."></DIV><DIV classes="email-body" ref="18" text="Condimentum. Et.."></DIV></DIV><DIV classes="email-right" ref="19"><DIV classes

In [ ]:
import json

In [ ]:
# Get clean episodes and randomize the refs

rd_history_episodes = []
rd_html_snippets = []
rd_instruction = []
rd_actions = []
rd_refs = []
rd_keydown_texts = []

# Keep track of some metrics
failed_lengths = []
empty_clean = []
failed_random = []
error_rows = []

# Now loop for all
i = 0
for row in dataset['train']:

  if i % 100 == 0:
    print(i)
  try:
    history_episodes, instruction, html_snippets, actions, refs, keydown_texts = clean_episode2(row)
  except Exception as e:
      print(f'{i} - {e}')
      error_rows.append(row)
      i += 1
      continue
  if len(actions) > 0:
    if len(history_episodes) == len(html_snippets) and len(html_snippets) == len(actions) and len(refs) == len(keydown_texts):
      new_history_episodes, new_html_snippets, new_instruction, new_actions, new_refs, new_keydown_texts = randomize_refs(history_episodes, instruction, html_snippets, actions, refs, keydown_texts)

      if len(new_history_episodes) > 0 and len(new_history_episodes) == len(new_html_snippets) and len(new_html_snippets) == len(new_keydown_texts) and len(new_actions) == len(new_refs):

        # All episodes are put individually here
        rd_history_episodes.append(new_history_episodes)
        rd_html_snippets.append(new_html_snippets)
        rd_instruction.append(new_instruction)
        rd_actions.append(new_actions)
        rd_refs.append(new_refs)
        rd_keydown_texts.append(new_keydown_texts)
      else:
        failed_random.append(row)
    else:
      failed_lengths.append(row)
  else:
    empty_clean.append(row)

  i += 1

print(f'failed_lengths {len(failed_lengths)} empty_clean = {len(empty_clean)} failed_random = {len(failed_random)}, error_rows = {len(error_rows)}')
len(rd_history_episodes), len(rd_html_snippets), len(rd_instruction), len(rd_actions), len(rd_refs), len(rd_keydown_texts)

0
100
200
204 - list index out of range
207 - list index out of range
300
354 - list index out of range
356 - list index out of range
359 - list index out of range
364 - list index out of range
370 - list index out of range
377 - list index out of range
400
413 - list index out of range
425 - list index out of range
430 - list index out of range
435 - list index out of range
500
547 - list index out of range
553 - list index out of range
565 - list index out of range
569 - list index out of range
573 - list index out of range
596 - list index out of range
600
607 - list index out of range
621 - list index out of range
622 - list index out of range
626 - list index out of range
631 - list index out of range
633 - list index out of range
700
763 - list index out of range
769 - list index out of range
772 - list index out of range
777 - list index out of range
782 - list index out of range
785 - list index out of range
800
840 - list index out of range
847 - list index out of range
855 - 

(12565, 12565, 12565, 12565, 12565, 12565)

In [ ]:
print(len(error_rows))

471


In [ ]:
# Some HTML Processing functions



# We remove all elements not needed in the received html to have it cleaned
def clean_html(html):
    tag_remove = ['script', 'img', 'meta', 'head', '<head', 'html', 'link', 'title', 'line', '!--', '!---']
    parts = html.replace('  ', '').split('\n')
    clean_html = ""
    for p in parts:
        if '<' in p:
            # check for tag
            tag = p.split('<')[1].split('>')[0].replace('/', '').split(' ')[0]
            if not tag in tag_remove:
                #print(tag)
                clean_html += p

    return clean_html

drop_fields = ['children', 'tag', 'onmousemove', 'onclick', 'onscroll', 'ontouchmove', 'ontouchstart', 'ontouchend', 'onkeypress', 'children', 'bgColor', 'width', 'height', 'focused', 'fgColor', 'right', 'top', 'left', 'bottom', 'class', 'classes']

# Converts the html to a string
def html2string(element):
    html_conct = "<" + element['tag']

    # add fields
    for field in element:
        if not field in drop_fields and field != "text":
            html_conct += ' ' + field + '="' + element[field] + '"'
    html_conct += '>'

    # add text
    html_conct += element['text']

    # If the element doesn't match, recursively search its children
    if "children" in element:
        for i, child in enumerate(element["children"]):
            path = html2string(child)
            html_conct += path

    html_conct += '</' + element['tag'] + '>'

    return html_conct

def build_tree(element):
        result = {"tag": element.tag}

        # Adding attributes as key-value pairs to the element
        if element.attrib:
            result.update({k: v for k, v in element.attrib.items()})

        # Adding text
        text = element.text.strip() if element.text else ""
        result['text'] = text

        # Adding child elements as nested dictionaries
        if len(element) > 0:
            result["children"] = [build_tree(child) for child in element]

        return result

from lxml import etree
parser = etree.HTMLParser()
def html2dict(html_str):
    tree = etree.fromstring(html_str, parser)# Starting from the root element and building the tree
    root = tree.getroottree().getroot()
    result = build_tree(root)
    return result

# Find whether <canvas> appears before or after the wrap
def wrapCanvasDist(html):
    canvas = len(html.split('<canvas')[0])
    wrap = len(html.split('id="wrap')[0])
    return wrap < canvas


def find_element_path2(element, attributes):
    # Check if the current element matches the attributes
    if all(attr in element.items() for attr in attributes.items()):
        return [element["tag"]]

    # If the element doesn't match, recursively search its children
    if "children" in element:
        for i, child in enumerate(element["children"]):
            path = find_element_path2(child, attributes)
            if path is not None:
                #print(f'element: {element["tag"]}, i:{i}, path:{path}')
                #return [element["tag"] , f"{i+1}/{len(element['children'])}"] + path
                return [element["tag"] , f"{i+1}"] + path
    # If no match was found, return None
    return None

# Returns the html element from the tree with its attributes
def find_element_by_ref(html_dict, ref_value):
    if "ref" in html_dict and html_dict["ref"] == ref_value:
        return html_dict
    elif "children" in html_dict:
        for child in html_dict["children"]:
            result = find_element_by_ref(child, ref_value)
            if result is not None:
                return result
    return None



In [ ]:
import time
i = 0

# Keeps tracks of the amount of episodes with a single action instead of multiples
unique_for_mutiple = 0
# Flight non optimal formats
flights_non_optimal = 0

# Keeps track of all processed elements
episodes_hierarchy = []

# all the rows that were not yet treated
different_from_submit = []
while i < len(rd_history_episodes):
  instruction = rd_instruction[i]
  ep_actions = rd_actions[i]
  ep_refs = rd_refs[i]
  episode = rd_history_episodes[i]
  snippets = rd_history_episodes[i]
  keydown_text = rd_keydown_texts[i]

  steps_in_episode = len(ep_actions)

  # List containing the individual instruction for each step
  instruction_steps = []
  # Eeach sub instruction can have multiple steps and refs
  action_steps = []
  ref_steps = []

  # Values for that episode
  episode_data = {}

  # Below is a match regarding the submit phase:
  if 'and click submit' in instruction or 'submit' in instruction.lower():

    # a) Deal with first part of instruction field
    instruction_p1 = instruction.split(' and')[0]

    # Need to figure out how many actions are present for instruction_p1:
    steps_p1 = steps_in_episode-1

    # Based on that, we should try to divide the instruction_p1 in these subtasks, or keep it fully
    if steps_in_episode == 2: # amount of actions in episode
      instruction_steps.append(instruction_p1) # append the first actions
      #action_steps.append([ep_actions[0]])
      #ref_steps.append([ep_refs[0]])
    elif steps_in_episode == 1:

      # In this case, we only do the only action we got and discard the submit button data
      if 'lect all the shad' in instruction:
        instruction_steps.append(instruction.split('and')[0])
        episode_data['instruction_steps'] = instruction_steps
        episode_data['instruction'] = instruction
        episode_data['actions'] = ep_actions
        episode_data['refs'] = ep_refs
        episode_data['text'] = keydown_text
        episode_data['episodes'] = rd_history_episodes[i]
        episode_data['html_snippets'] = rd_html_snippets[i]
        episodes_hierarchy.append(episode_data)

        i += 1
        continue
      elif 'Select nothing and click Submit' in instruction:
        instruction_steps.append(instruction)
        episode_data['instruction_steps'] = instruction_steps
        episode_data['instruction'] = instruction
        episode_data['actions'] = ep_actions
        episode_data['refs'] = ep_refs
        episode_data['text'] = keydown_text
        episode_data['episodes'] = rd_history_episodes[i]
        episode_data['html_snippets'] = rd_html_snippets[i]
        episodes_hierarchy.append(episode_data)
        i += 1
        continue

      # If we have only one step for a click submit action, it means that something is wrong and we should skip it entirely
      i += 1
      unique_for_mutiple += 1
      print(f'{i} - ØØØØ unique for multiple: {instruction} {ep_actions} {ep_refs} {snippets}')
      continue
    else:

      # Enter time experiment
      if 'as the time and press submit' in instruction:
        instruction_p1 = instruction.split(' and press')[0]
        # Add actions until submit part
        if steps_in_episode == 3:
          instruction_steps.append(instruction_p1) # append the first actions
          instruction_steps.append(instruction_p1)
        else:
          print(f'{i} - Got {steps_in_episode} for enter time: {ep_actions}, {ep_refs}, {keydown_text}')
          i += 1
          continue
      elif 'as the date and hit submit' in instruction:
        instruction_p1 = instruction.split(' and hit')[0]
        # Add actions until submit part
        if steps_in_episode == 3:
          instruction_steps.append(instruction_p1) # append the first actions
          instruction_steps.append(instruction_p1)
        else:
          print(f'Got {steps_in_episode} for enter date: {ep_actions}, {ep_refs}, {keydown_text}')
          i += 1
          continue

      else:

        if ('nto the text field and press Sub' in instruction and 'Enter' in instruction and len(ep_actions) == 3):# or ('Enter' in instruction and 'into the text field and press Submi' in instruction):
          if ep_actions[0] == 'click' and ep_actions[1] == 'keydown':
            instruction_p1 = instruction.split('and press')[0]
            instruction_steps.append(instruction_p1)
            instruction_steps.append(instruction_p1)
            instruction_steps.append('press submit')

            episode_data['instruction_steps'] = instruction_steps
            episode_data['instruction'] = instruction
            episode_data['actions'] = ep_actions
            episode_data['refs'] = ep_refs
            episode_data['text'] = keydown_text
            episode_data['episodes'] = rd_history_episodes[i]
            episode_data['html_snippets'] = rd_html_snippets[i]
            episodes_hierarchy.append(episode_data)
        elif 'Enter' in instruction and 'nto both text fields and pres' in instruction:
          #Enter the password "Z5X" into both text fields and press submit.
          # have either lengths of 4 or 5
          new_actions = [ep_actions[0], ep_actions[1], ep_actions[2]]
          new_refs = [ep_refs[0], ep_refs[1], ep_refs[2]]
          new_texts = [keydown_text[0], keydown_text[1], keydown_text[2]]
          new_instruction = instruction.split('into')[0]
          instruction_steps.append(new_instruction + 'into the first text field')
          instruction_steps.append(new_instruction + 'into the first text field')
          instruction_steps.append(new_instruction + 'into the second text field')
          if len(ep_actions) == 5:
            new_actions.append(ep_actions[3])
            new_refs.append(ep_refs[3])
            new_texts.append(keydown_text[3])
            instruction_steps.append(new_instruction + 'into the second text field')
            new_actions.append(ep_actions[4])
            new_refs.append(ep_refs[4])
            new_texts.append(keydown_text[4])
            instruction_steps.append('Press Submit')
            episodes_hierarchy.append(episode_data)
          elif len(ep_actions)== 4:
            new_actions.append(ep_actions[3])
            new_refs.append(ep_refs[3])
            new_texts.append(keydown_text[3])
            instruction_steps.append('Press Submit')
            episodes_hierarchy.append(episode_data)
          else:
            print(f'{i} - error, different length than 4 or 5: {instruction}, {ep_actions}')
            i += 1
            continue

        elif 'Select ' in instruction and len(ep_actions) >= 3:
          instruction_parts = instruction.split(' and')[0].replace('Select ', '').split(',')
          #if len(instruction_parts) != len(ep_actions)-1:
          #  print(f'{i} - not matching lengths of instructions and actions: {instruction}, {ep_actions}')
          new_actions = []
          new_refs = []
          new_texts = []
          j = 0
          # add all remaining actions:
          for p in instruction_parts:
            if j > len(ep_actions)-2:
              break
            instruction_steps.append('Select ' + p)
            new_actions.append(ep_actions[j])
            new_refs.append(ep_refs[j])
            new_texts.append(keydown_texts[j])
            j += 1
          # add submit
          new_actions.append(ep_actions[len(ep_actions)-1])
          new_refs.append(new_refs[len(new_refs)-1])
          new_texts.append(new_texts[len(new_texts)-1])
          instruction_steps.append('click submit')
          episode_data['instruction_steps'] = instruction_steps
          episode_data['instruction'] = instruction
          episode_data['actions'] = new_actions
          episode_data['refs'] = new_refs
          episode_data['text'] = new_texts
          episode_data['episodes'] = rd_history_episodes[i]
          episode_data['html_snippets'] = rd_html_snippets[i]
          episodes_hierarchy.append(episode_data)
        else:
          print(f'{i} - ØØØ Got more steps instruction: "{instruction}", actions: "{ep_actions}", refs: {ep_refs} keydown_text: {keydown_text}')
          i += 1
          continue

    # b) Deal with Submit action by getting the correct ref element
    # This ref number should be the last action:
    submit_ref = ep_refs[len(ep_refs)-1]
    #action_steps.append([ep_actions[len(ep_actions)-1]])
    #ref_steps.append([ep_refs[len(ep_refs)]])

    # Add the submit action as the last one, and add the full action history first too.
    # ? ==> Need to figure out wether submit should be part of the initial instruction or not, it needs to come in at some point
    instruction_steps.append('click submit')

    episode_data['instruction_steps'] = instruction_steps
    episode_data['instruction'] = instruction
    episode_data['actions'] = ep_actions
    episode_data['refs'] = ep_refs
    episode_data['text'] = keydown_text
    episode_data['episodes'] = rd_history_episodes[i]
    episode_data['html_snippets'] = rd_html_snippets[i]
    episodes_hierarchy.append(episode_data)

  elif '{' in instruction:
    # Deal with the flights
    # a) parse the instruction json
    flight_instruction = json.loads(instruction)

    if len(ep_actions) < 4:
      i += 1
      flights_non_optimal += 1
      print(f'ØØØ Non optimak flight, LENGTH {instruction}, {ep_actions}')
      continue

    # Check how many actions we have, and check the tuples of click + keydown.
    # They will match with selecting a box and inputing text inside.
    # Usually it is departure, then destination. + a date
    # We should modify the instructions for each of these sub actions.

    # Get actions
    # 1) Departure
    if ep_actions[0] == 'click' and ep_actions[1] == 'keydown':
      instruction_departure = "Select Departure City " + flight_instruction['Departure City']
      steps_departure = [ep_actions[0], ep_actions[1]]
      refs_departure = [ep_refs[0], ep_refs[1]]
      instruction_steps.append(instruction_departure)
      instruction_steps.append(instruction_departure)
    else:
      i += 1
      flights_non_optimal += 1
      print(f'{i} - w1 {ep_actions}')
      continue

    # 2) Destination
    if ep_actions[2] == 'click' and ep_actions[3] == 'keydown':
      instruction_departure = "Select Destination City " + flight_instruction['Destination City']
      steps_departure = [ep_actions[2], ep_actions[3]]
      refs_departure = [ep_refs[2], ep_refs[3]]
      instruction_steps.append(instruction_departure)
      instruction_steps.append(instruction_departure)
    else:
      i += 1
      flights_non_optimal += 1
      print(f'w1 {ep_actions}')
      continue

    # 3) Other fields such as day, amount of passengers etc
    # just take the remaining actions

    j = 4
    while j < len(ep_actions):
      # Get the html dictionary with the different attributes to match the actions
      clean_html_dict = html2dict(snippets[j])

      # Get ref number and match it to the element it is targeting to understand what action is being done
      element = find_element_by_ref(clean_html_dict, str(ep_refs[j]))

      #print(element)

      # Figure out what is the purpose of the action
      if element.get('classes') is not None:
        if 'day' in element['classes']:
          instruction_plus = "Select the Departure Day to " + str(flight_instruction['Departure Day'])
        elif 'cabin' or 'seat' in element['classes']:
          instruction_plus = "Select the Seat type to " + flight_instruction['Seat type']
        else:
          print(f'{i} - got unmatched for {flight_instruction}, action: {ep_actions[j]}, {clean_html_dict}')
          i += 1
          break
      elif element.get('id') is not None:
        if 'day' in element['id']:
          instruction_plus = "Select the Departure Day to " + str(flight_instruction['Departure Day'])
        elif 'cabin' or 'seat' in element['id']:
          instruction_plus = "Select the Seat type to " + flight_instruction['Seat type']
        else:
          print(f'{i} - got unmatched for {flight_instruction}, action: {ep_actions[j]}, {clean_html_dict}')
          i += 1
          break
      else:
        print(f'{i} - got unmatched for {flight_instruction}, action: {ep_actions[j]}, {clean_html_dict}')

      j += 1

      instruction_steps.append(instruction_plus)

    episode_data['instruction_steps'] = instruction_steps
    episode_data['instruction'] = instruction
    episode_data['actions'] = ep_actions
    episode_data['refs'] = ep_refs
    episode_data['text'] = keydown_text
    episode_data['episodes'] = rd_history_episodes[i]
    episode_data['html_snippets'] = rd_html_snippets[i]
    episodes_hierarchy.append(episode_data)


  elif "Click" in instruction:
    instruction_steps.append(instruction)

    episode_data['instruction_steps'] = instruction_steps
    episode_data['instruction'] = instruction
    episode_data['actions'] = ep_actions
    episode_data['refs'] = ep_refs
    episode_data['text'] = keydown_text
    episode_data['episodes'] = rd_history_episodes[i]
    episode_data['html_snippets'] = rd_html_snippets[i]
    episodes_hierarchy.append(episode_data)

  elif "Switch between the tabs" in instruction:
    # First instruction is to switch tabs to look for a specific link, second is to click on that link
    # eg: "Switch between the tabs to find and click on the link "risus". "
    target_link = instruction.split('"')[1]
    instruction_p1 = 'Switch between the tabs to find the link "' + target_link + '".'
    instruction_p2 = 'Click on the link "' + target_link + '".'

    j = 0
    while j < len(ep_actions)-1:
      instruction_steps.append(instruction_p1)
      j += 1
    instruction_steps.append(instruction_p2)

    episode_data['instruction_steps'] = instruction_steps
    episode_data['instruction'] = instruction
    episode_data['actions'] = ep_actions
    episode_data['refs'] = ep_refs
    episode_data['text'] = keydown_text
    episode_data['episodes'] = rd_history_episodes[i]
    episode_data['html_snippets'] = rd_html_snippets[i]
    episodes_hierarchy.append(episode_data)

  elif "mail" in instruction or "message" in instruction:
    j = 0
    while j < len(ep_actions):
      instruction_steps.append(instruction)
      j += 1

    episode_data['instruction_steps'] = instruction_steps
    episode_data['instruction'] = instruction
    episode_data['actions'] = ep_actions
    episode_data['refs'] = ep_refs
    episode_data['text'] = keydown_text
    episode_data['episodes'] = rd_history_episodes[i]
    episode_data['html_snippets'] = rd_html_snippets[i]
    episodes_hierarchy.append(episode_data)

  elif "Enter the username" in instruction:
    # Three step process of getting username, password and then pressing login
    instruction_parts_index = 0
    instruction_parts = instruction.split(' and')
    print(instruction_parts)
    instruction_username = instruction_parts[0]
    instruction_password = 'Enter' + instruction_parts[1].split('into')[0]
    instruction_login = 'press login'

    j = 0
    step_p = 0
    while j < len(ep_actions):
      if step_p == 0:
        instruction_steps.append(instruction_username)
      elif step_p == 1:
        instruction_steps.append(instruction_password)
      else:
        instruction_steps.append(instruction_login)
      if ep_actions[j] == 'keydown':
        step_p += 1
      j += 1

    episode_data['instruction_steps'] = instruction_steps
    episode_data['instruction'] = instruction
    episode_data['actions'] = ep_actions
    episode_data['refs'] = ep_refs
    episode_data['text'] = keydown_text
    episode_data['episodes'] = rd_history_episodes[i]
    episode_data['html_snippets'] = rd_html_snippets[i]
    episodes_hierarchy.append(episode_data)

  elif "Enter an item that" in instruction:
    # trim to have two actions in instruction_1, and one for instruction_2 (submit)
    new_actions = []
    new_refs = []
    new_text = []
    if ep_actions[0] == 'click' and ep_actions[1] == 'keydown' and len(ep_actions) >= 3:
      new_actions.append(ep_actions[0])
      new_refs.append(ep_refs[0])
      new_text.append(keydown_text[0])
      instruction_steps.append(instruction) # No need to split as no submit is in there
      new_actions.append(ep_actions[1])
      new_refs.append(ep_refs[1])
      new_text.append(keydown_text[1])
      instruction_steps.append(instruction)

      # add submit actions
      new_actions.append(ep_actions[len(ep_actions)-1])
      new_refs.append(ep_refs[len(ep_refs)-1])
      new_text.append(keydown_text[len(keydown_text)-1])
      instruction_steps.append('click submit')

      new_episodes = [rd_history_episodes[i][0], rd_history_episodes[i][1], rd_history_episodes[i][len(rd_history_episodes[i])-1]]
      new_snippets = [rd_html_snippets[i][0], rd_html_snippets[i][1], rd_html_snippets[i][len(rd_html_snippets[i])-1]]

      episode_data['instruction_steps'] = instruction_steps
      episode_data['instruction'] = instruction
      episode_data['actions'] = new_actions
      episode_data['refs'] = new_refs
      episode_data['text'] = new_text
      episode_data['episodes'] = new_episodes
      episode_data['html_snippets'] = new_snippets
      episodes_hierarchy.append(episode_data)

    else:
      print(f'{i} - wrong order of actions for {instruction}, {ep_actions}')
      i += 1
      continue

  elif "Use the textbox to enter" in instruction:
    new_actions = []
    new_refs = []
    new_text = []
    # Use the textbox to enter "Kasie" and press "Search", then find and click the 6th search result.
    instruction_textbox = instruction.split(' and')[0]
    instruction_search = 'press "Search"'
    instruction_click = instruction.split('then ')[1]

    # only take optimal with 4 actions:
    if len(ep_actions) == 4:
      if ep_actions[1] == 'keydown':
        new_actions.append(ep_actions[0])
        new_refs.append(ep_refs[0])
        new_text.append(keydown_text[0])
        new_actions.append(ep_actions[1])
        new_refs.append(ep_refs[1])
        new_text.append(keydown_text[1])
        instruction_steps.append(instruction_textbox)
        instruction_steps.append(instruction_textbox)

        new_actions.append(ep_actions[2])
        new_refs.append(ep_refs[2])
        new_text.append(keydown_text[2])
        instruction_steps.append('press "Search"')

        new_actions.append(ep_actions[3])
        new_refs.append(ep_refs[3])
        new_text.append(keydown_text[3])
        instruction_steps.append(instruction.split('then ')[1])

        new_episodes = [rd_history_episodes[i][0], rd_history_episodes[i][1], rd_history_episodes[i][2], rd_history_episodes[i][3]]
        new_snippets = [rd_html_snippets[i][0], rd_html_snippets[i][1], rd_history_episodes[i][2], rd_history_episodes[i][3]]

        episode_data['instruction_steps'] = instruction_steps
        episode_data['instruction'] = instruction
        episode_data['actions'] = new_actions
        episode_data['refs'] = new_refs
        episode_data['text'] = new_text
        episode_data['episodes'] = new_episodes
        episode_data['html_snippets'] = new_snippets
        episodes_hierarchy.append(episode_data)

      else:
        i += 1
        continue
    if len(ep_actions) > 4:
      # Just take the first action of selection, and the last one of selecting, but not the middleone of pressing search as we can't garantee what these actions do
      if ep_actions[1] == 'keydown':
        new_actions.append(ep_actions[0])
        new_refs.append(ep_refs[0])
        new_text.append(keydown_text[0])
        new_actions.append(ep_actions[1])
        new_refs.append(ep_refs[1])
        new_text.append(keydown_text[1])
        instruction_steps.append(instruction_textbox)
        instruction_steps.append(instruction_textbox)
        new_episodes = [rd_history_episodes[i][0], rd_history_episodes[i][1]]
        new_snippets = [rd_html_snippets[i][0], rd_html_snippets[i][1]]

        # Skip press Enter and get the last one directly
        episode_data['instruction_steps'] = instruction_steps
        episode_data['instruction'] = instruction
        episode_data['actions'] = new_actions
        episode_data['refs'] = new_refs
        episode_data['text'] = new_text
        episode_data['episodes'] = new_episodes
        episode_data['html_snippets'] = new_snippets
        episodes_hierarchy.append(episode_data)
      else:
        i += 1
        continue
    else:
      i += 1
      continue
  elif 'xpand the sections below, to find and click on the ' in instruction:
    instruction_steps.append(instruction.split(',')[0])
    instruction_steps.append(instruction.split('to find and')[1])
    new_actions = [ep_actions[0], ep_actions[len(ep_actions)-1]]
    new_refs = [ep_refs[0], ep_refs[len(ep_refs)-1]]
    keydowns = [keydown_text[0], keydown_text[len(keydown_text)-1]]
    new_episodes = [rd_history_episodes[i][0], rd_history_episodes[i][len(rd_history_episodes[i])-1]]
    new_snippets = [rd_html_snippets[i][0], rd_html_snippets[i][len(rd_html_snippets[i])-1]]
    episode_data['instruction_steps'] = instruction_steps
    episode_data['instruction'] = instruction
    episode_data['actions'] = new_actions
    episode_data['refs'] = new_refs
    episode_data['text'] = keydowns
    episode_data['episodes'] = new_episodes
    episode_data['html_snippets'] = new_snippets
    episodes_hierarchy.append(episode_data)

  elif 'Close the dialog box by clicking the "x"' in instruction:
    instruction_steps.append(instruction)
    episode_data['instruction_steps'] = instruction_steps
    episode_data['instruction'] = instruction
    episode_data['actions'] = ep_actions
    episode_data['refs'] = ep_refs
    episode_data['text'] = keydown_text
    episode_data['episodes'] = rd_history_episodes[i]
    episode_data['html_snippets'] = rd_html_snippets[i]
    episodes_hierarchy.append(episode_data)

  else:
    print(f'{i} - Different than submit: instruction: "{instruction}", actions: {ep_actions}, keydowns: {keydown_text} refs: {ep_refs}, snippet: {rd_html_snippets[i]}')

    # For all weird encounters, we add defaults
    instruction_steps.append(instruction)
    episode_data['instruction_steps'] = instruction_steps
    episode_data['instruction'] = instruction
    episode_data['actions'] = ep_actions
    episode_data['refs'] = ep_refs
    episode_data['text'] = keydown_text
    episode_data['episodes'] = rd_history_episodes[i]
    episode_data['html_snippets'] = rd_html_snippets[i]
    episodes_hierarchy.append(episode_data)


    i += 1
    continue

  # Print what was found at that step
  #print(f"{i}- instruction_steps:{episode_data['instruction_steps']}, actions: {episode_data['actions']}, refs: {episode_data['refs']}, keydown_text: {episode_data['text']}, snippet: {episode_data['html_snippets']}")
  #print('---------------------------------------------------------------------------')

  i += 1

  if i / 500 == 0:
    time.sleep(5)

  if i>7000:
    break

print(f'found {unique_for_mutiple} of unique_for_mutiple actions that were discarded')
print(f'found total of {flights_non_optimal} flights_non_optimal')
print(f'found total {len(episodes_hierarchy)} episodes_hierarchy')

4 - ØØØØ unique for multiple: Expand the section below and click submit.  ['click'] [3] ['Expand the section below and click submit.  <body ref="7"><div id="wrap" ref="4"><div id="area" classes="ui-accordion ui-widget ui-helper-reset" ref="6"><h3 id="ui-id-4" classes="ui-accordion-header ui-corner-top ui-state-default ui-accordion-icons ui-accordion-header-active ui-state-active" ref="5"><t ref="-12" text="section #16"></t></h3><div id="ui-id-5" classes="ui-accordion-content ui-corner-bottom ui-helper-reset ui-widget-content ui-accordion-content-active" ref="2" text="scelerisque morbi accumsan, facilisis ac sit purus euismod hac eu morbi euismod bibendum mauris. a sed tortor purus porttitor morbi."></div><p id="ui-id-6" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons ui-state-hover ui-state-focus" ref="1"><button id="subbtn" classes="secondary-action" ref="3" text="submit"></button></p></div></div></body>']
5 - 

In [ ]:
print(f'found {len(different_from_submit)} different_from_submit')

found 0 different_from_submit


In [ ]:
index = 924
rd_html_snippets[index], rd_instruction[index], rd_refs[index], rd_actions[index], rd_keydown_texts[index]

(['<body ref="12"><form id="mturk_form" ref="7"><div id="wrap" ref="8"><div id="area" ref="2"><div id="forward" ref="15"><div id="forward-bar" ref="5"><span id="close-forward" ref="13"></span><span id="send-forward" ref="6"></span></div><div classes="forward-header" ref="9"><div classes="forward-info" ref="1"><label ref="10" text="to:"></label><input_text classes="forward-sender" ref="11"></input_text></div><div classes="forward-subject" ref="3"><label ref="4" text="subject:"></label><t ref="-6" text="dwelling."></t></div></div><div classes="forward-body" ref="14"><textarea id="forward-text" ref="16" value="u.s.-backed tens ancient defected cliffs. 350 nec reprisals aspects ferreira by exiles commuter."></textarea></div></div></div></div></form></body>, ',
  '<body ref="12"><form id="mturk_form" ref="7"><div id="wrap" ref="8"><div id="area" ref="2"><div id="forward" ref="15"><div id="forward-bar" ref="5"><span id="close-forward" ref="13"></span><span id="send-forward" ref="6"></span></

In [ ]:
import json

In [ ]:
flight_instruction = json.loads(rd_instruction[index])
flight_instruction

In [ ]:
clean_html_dict = html2dict(rd_html_snippets[index][4])
clean_html_dict

In [ ]:


r= find_element_by_ref(clean_html_dict, "152")
r

In [ ]:
len(rd_html_snippets)

0

# More exploration

In [ ]:

from datasets import load_dataset

# Load dataset regarding action-step sequences
raw_datasets_actions = load_dataset("LucasThil/miniwob_plusplus_hierarchical_training_actions")

# Load dataset regarding task of hierarchical planning
raw_datasets_planning = load_dataset("LucasThil/miniwob_plusplus_hierarchical_planning")



Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/42097 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/LucasThil___parquet/LucasThil--miniwob_plusplus_hierarchical_training_actions-49f035c84c4bf8ff/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10960 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/LucasThil___parquet/LucasThil--miniwob_plusplus_hierarchical_planning-55282b447acace9c/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:

import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
    return df


In [ ]:
r = show_random_elements(raw_datasets_actions['train'])


# CC-Net
We are preparing a new dataset in order to generate the corresponding images. For that we need a full a complete html code instead of snippets and performing actions on them to remain as much accurate as possible.

In [ ]:
import traceback

# Loop do for the entire dataset, get episodes together

cleaned_episodes = []

i = 0
error_rows = [] # Errors when trying to clean_episode2
error_empty = []  # Errors when retrieving actions lengths being 0

for row in dataset['train']:
  if i % 100 == 0:
    print(i)
  try:
    history_episodes_r, instruction_r, html_snippets_r, actions_r, refs_r, keydown_texts_r = clean_episode2(row)
  except Exception as e:
    error_rows.append(row)
    i += 1
    continue
  if len(actions_r) > 0:
    if len(history_episodes_r) == len(html_snippets_r) and len(html_snippets_r) == len(actions_r) and len(refs_r) == len(keydown_texts_r):
      cleaned_episode = {
          'history_episodes_r': history_episodes_r,
          'instruction_r': instruction_r,
          'html_snippets_r': html_snippets_r,
          'actions_r': actions_r,
          'refs_r': refs_r,
          'keydown_texts_r': keydown_texts_r
      }
      cleaned_episodes.append(cleaned_episode)
    else:
      print(f"Error processing, not equal lengths at index {i}: {row}")
      break
  else:
    #print(f'No action length found: {actions_r}, row: {row}')
    error_empty.append(row)
  i += 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700


In [ ]:
cleaned_episodes[0]

{'history_episodes_r': ['Expand the section below and click submit.  <body ref="1"><div id="wrap" ref="2"><div id="area" classes="ui-accordion ui-widget ui-helper-reset" ref="3"><h3 id="ui-id-1" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons ui-state-hover ui-state-focus" ref="4"><t ref="-6" text="section #9"></t></h3><p id="ui-id-3" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons" ref="5"><button id="subbtn" classes="secondary-action" ref="6" text="submit"></button></p></div></div></body>, ',
  '{click,4,} Expand the section below and click submit.  <body ref="1"><div id="wrap" ref="2"><div id="area" classes="ui-accordion ui-widget ui-helper-reset" ref="3"><h3 id="ui-id-1" classes="ui-accordion-header ui-corner-top ui-state-default ui-accordion-icons ui-accordion-header-active ui-state-active" ref="4"><t ref="-12" text="section #9"></t>

Now we have to find out, which task each corresponds to. For that we create a table of the different miniwob tasks and instruction format.

In [ ]:
tasks = {
    'bisect-angle': ['Create a line that bisects the angle evenly in two, then press submit.'],
    'book-flight': ['Book the', 'one-way flight from', 'on'],
    'book-flightnodelay': ['"Departure City"'],
    'choose-date': ['Select', 'as the date and hit submit'],
    'choose-list': ['Select', 'from the list and click submit'],
    'click-button-sequence': ['Click button ONE, then click button TWO.'],
    'click-checkboxes-soft': ['Select words similar to', 'and click Submit.'],
    'click-checkboxes': ['Select', ',', 'and click Submit.'],
    'click-checkboxes-transfer': ['Select', 'and click Submit.'],
    'click-collapsible-2': ['Expand the sections below, to find and click on the link'],
    'click-collapsible': ['Expand the section below and click submit.'],
    'click-color': ['Click on the',  'colored box.'],
    'click-dialog-2': ['Click the button in the dialog box labeled'],
    'click-dialog': ['Close the dialog box by clicking the'],
    'click-link': ['Click on the link'],
    'click-menu-2': ['Click the', 'and then find and click on the item with the'],
    'click-menu': ['Select', '>'],
    'click-option': ['Select', 'and click Submit'],
    'click-pie': ['Expand the pie menu below and click on the item labeled'],
    'click-scroll-list': ['Select', 'from the scroll list and click Submit.'],
    'click-shades': ['Select all the shades of'],
    'click-shape': ['Click on a'],
    'click-tab-2': ['Switch between the tabs to find and click on the link'], # can have different amount of tables according to the hardness levels
    'click-tab': ['Click on Tab'],
    'click-test-2': ['Click button ONE.'],
    'click-test-transfer': ['Click button ONE.'],
    'click-test': ['Click the button.'],
    'click-widget': ['Click on a', 'widget.'],
    'copy-paste-2': ['Copy the text from the', 'text area below and paste it into the text input, then press Submit.'],
    'copy-paste': ['Copy the text in the textarea below, paste it into the textbox and press Submit.'],
    'count-shape': ['How many ', 'are there?'],
    'count-sides': ['Press the button that correctly denotes how many sides the shape has.'],
    'drag-box': ['Drag the', 'box'],
    'drag-cube': ['Move the cube around'],
    'drag-item': ['Drag the', 'and then press Submit.'],
    'drag-items-grid': ['Drag', 'to the'],
    'drag-items': ['Drag', 'position'],
    'drag-shapes': ['Drag all', 'into'],
    'drag-sort-numbers': ['Sort the numbers in'],
    'email-inbox': ['mail'],
    'message': ['message'],
    'email-inbox-delete': ['he trash icon to delete it'],
    'email-inbox-forward-nl-turk': ['Forward', 'email'],
    'email-inbox-forward': ['Navigate to the message', 'and send it'], ### Deal with emails later

    'enter-date': ['Enter', 'as the date and hit submit.'],
    'enter-password': ['Enter the password', 'into both text fields and press submit.'],
    'enter-text-2': ['Type', 'case letters in the text input and press Submit.'],
    'enter-text': ['Enter', 'into the text field and press Submit.'],
    'enter-time': ['Enter', 'as the time and press submit.'],
    'find-midpoint': ['Find and click', 'mid-point'],
    'find-word': ['Find the', 'in the paragraph, type that into the textbox and press "Submit"'],
    'focus-text': ['Focus into the textbox.'],
    'focus-text-2': ['Focus into the', 'textbox'],
    'grid-coordinate': ['Click on the grid coordinate'],
    'guess-number': ['Guess the number between'],
    'highlight-text-2': ['Highlight the text in the', 'paragraph and click submit.'],
    'highlight-text': ['Highlight the text in the paragraph below and click submit.'],
    'identify-shape': ['Click the button that best describes the figure below.'],
    'login-user': ['Enter the username', 'into the text fields and press login.'],
    'moving-items': ['Click as many moving circles as possible.'],
    'multi-layouts': ['Search for', 'movies', 'directed by', 'from'],
    'navigate-tree': ['Navigate through the file tree.'],
    'number-checkboxes': ['in the checkboxes using the example on the right and press Submit when finished.'],
    'read-table-2': ['Enter the value that corresponds with each label into the form and submit when done'],
    'resize-textarea': ['Resize the textarea so that the'],
    'right-angle': ['to create a right angle, then press submit.'],
    'scroll-text-2': ['Scroll the textarea to the bottom of the text hit submit.'],
    'scroll-text': ['Find the last word in the text area, enter it into the text field and hit Submit.'],
    'search-engine': ['Use the textbox to enter', 'and press "Search", then find and click the', 'search result.'],
    'simon-says': ['Push the buttons in the order displayed.'],
    'simple-algebra': ['Solve for x and type your answer into the textbox. Press Submit when done.'],
    'simple-arithmetic': ['Solve the math problem and type your answer into the textbox. Press submit when done.'],
    'social-media-all': ['Click the', 'button on all posts by', 'and then click'],
    'social-media-some': ['Click the', 'button on', 'posts by', 'and then click'],
    'social-media': ['For the user', 'click on the', 'button'],
    'terminal': ['Use the terminal below'],
    'text-editor': ['Using the text editor, give'],
    'text-transform': ['Type the text below into the text field and press Submit.'],
    'tic-tac-toe': ['win a game of tic-tac-toe.'],
    'unicode-text': ['Click on the "', '" button.'],
    'use-autocomplete': ['Enter an item that starts with'],
    'use-colorwheel-2': ['Select the following color', 'with the color picker and hit Submit.'],
    'use-colorwheel': ['with the color picker and hit Submit.'],
    'use-slider-2': ['Set the sliders to the combination', 'and submit.'],
    'use-slider': ['with the slider and hit Submit.'],
    'use-spinner': ['with the spinner and hit Submit. '],
}

In [ ]:
i = 0

while i < len(cleaned_episodes):
  row = cleaned_episodes[i]
  instruction = row['instruction_r']

  for task in tasks:
    found = True
    #print(f'task: {task}')

    for part in tasks[task]:
      #print(part)
      if not part in instruction:
        found = False
        break

    # Found one
    if found:
      # Add task to episode dictionary
      cleaned_episodes[i]['task'] = task
      break

  if not found:
    print(f'{i} - Not found for instruction {instruction}')

    # Add default for not found
    cleaned_episodes[i]['task'] = 'unknown'

    i += 1
    continue
  i += 1

  if i > 5000 and False:
    break


210 - Not found for instruction Send Almeria the information Ainslee sent to you. 
218 - Not found for instruction Click the "Retweet" button on 1 post by @myron and then click Submit. 
220 - Not found for instruction Click the "Share" button on 1 post by @kanesha and then click Submit. 
227 - Not found for instruction Click the "Like" button on 1 post by @emile and then click Submit. 
228 - Not found for instruction Click the "Share" button on 1 post by @michel and then click Submit. 
229 - Not found for instruction Click the "Share" button on 1 post by @michel and then click Submit. 
230 - Not found for instruction Click the "Reply" button on 1 post by @kasie and then click Submit. 
232 - Not found for instruction Click the "Like" button on 1 post by @livia and then click Submit. 
234 - Not found for instruction Click the "Reply" button on 1 post by @kenda and then click Submit. 
239 - Not found for instruction Click the "Reply" button on 1 post by @keli and then click Submit. 
240 -

In [ ]:
cleaned_episodes[0]

{'history_episodes_r': ['Expand the section below and click submit.  <body ref="1"><div id="wrap" ref="2"><div id="area" classes="ui-accordion ui-widget ui-helper-reset" ref="3"><h3 id="ui-id-1" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons ui-state-hover ui-state-focus" ref="4"><t ref="-6" text="section #9"></t></h3><p id="ui-id-3" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons" ref="5"><button id="subbtn" classes="secondary-action" ref="6" text="submit"></button></p></div></div></body>, ',
  '{click,4,} Expand the section below and click submit.  <body ref="1"><div id="wrap" ref="2"><div id="area" classes="ui-accordion ui-widget ui-helper-reset" ref="3"><h3 id="ui-id-1" classes="ui-accordion-header ui-corner-top ui-state-default ui-accordion-icons ui-accordion-header-active ui-state-active" ref="4"><t ref="-12" text="section #9"></t>

Now need to replace refs

In [ ]:

randomized_dataset = []

for row in cleaned_episodes:
  if len(randomized_dataset) % 100 == 0:
    print(len(randomized_dataset))

  history_episodes = row['history_episodes_r']
  instruction = row['instruction_r']
  html_snippets = row['html_snippets_r']
  actions = row['actions_r']
  refs = row['refs_r']
  keydown_texts = row['keydown_texts_r']

  new_history_episodes, new_html_snippets, new_instruction, new_actions, new_refs, new_keydown_texts = randomize_refs(history_episodes, instruction, html_snippets, actions, refs, keydown_texts)

  new_row = {
      'task': row['task'],
      'history_episodes': new_history_episodes,
      'html_snippets': new_html_snippets,
      'instruction': instruction,
      'actions': new_actions,
      'refs': new_refs,
      'keydown_texts': new_keydown_texts,
  }
  randomized_dataset.append(new_row)

len(randomized_dataset)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400


14494

In [ ]:
randomized_dataset[0]

{'task': 'click-collapsible',
 'history_episodes': ['Expand the section below and click submit.  <body ref="2"><div id="wrap" ref="3"><div id="area" classes="ui-accordion ui-widget ui-helper-reset" ref="6"><h3 id="ui-id-1" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons ui-state-hover ui-state-focus" ref="5"><t ref="-6" text="section #9"></t></h3><p id="ui-id-3" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons" ref="7"><button id="subbtn" classes="secondary-action" ref="4" text="submit"></button></p></div></div></body>, ',
  '{click,5,} Expand the section below and click submit.  <body ref="2"><div id="wrap" ref="3"><div id="area" classes="ui-accordion ui-widget ui-helper-reset" ref="6"><h3 id="ui-id-1" classes="ui-accordion-header ui-corner-top ui-state-default ui-accordion-icons ui-accordion-header-active ui-state-active" ref="5"><t ref=

In [ ]:
import pandas as pd
df = pd.DataFrame(randomized_dataset)
df.to_csv("/content/sample_data/train.csv", index=False)

In [ ]:
from datasets import load_dataset
file_dict = {
  "train" : "/content/sample_data/train.csv",
}
df_train = load_dataset("csv", data_files=file_dict)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-6646f5fc59c09433/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df_train

DatasetDict({
    train: Dataset({
        features: ['task', 'history_episodes', 'html_snippets', 'instruction', 'actions', 'refs', 'keydown_texts'],
        num_rows: 14494
    })
})

In [ ]:
df_train.push_to_hub('LucasThil/miniwob_plusplus_task_randomized_ccnet')

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

# Missing some text from original raw miniwob episodes

In [ ]:
!pip install --upgrade datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import html

from datasets import load_dataset

# Load dataset regarding action-step sequences
miniwob_dataset = load_dataset("LucasThil/miniwobplusplus")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: ignored

In [ ]:


def generate_html_code(data):
  # Generate the HTML code
  html_code = ''
  for item in data:
      dom = item['dom']
      tag = dom['tag'].lower()
      attrs = ''
      styles = ''
      for key, value in dom.items():
          if key in ['tag', 'children']:
              continue
          elif key == 'text':
              html_text = html.escape(value)
              html_code += f'<{tag}>{html_text}</{tag}>'
          elif key == 'classes':
              attrs += f' class="{value}"'
          elif key == 'id':
              attrs += f' id="{value}"'
          elif key == 'bgColor':
              styles += f' background-color:{value};'
          elif key == 'fgColor':
              styles += f' color:{value};'
          else:
              styles += f' {key}:{value}px;'
      html_code += f'<{tag}{attrs} style="{styles}">'
      if 'children' in dom:
          for child in dom['children']:
              child_tag = child['tag'].lower()
              child_attrs = ''
              child_styles = ''
              for key, value in child.items():
                  if key in ['tag', 'children']:
                      continue
                  elif key == 'text':
                      html_text = html.escape(value)
                      html_code += f'<{child_tag}>{html_text}</{child_tag}>'
                  elif key == 'classes':
                      child_attrs += f' class="{value}"'
                  elif key == 'id':
                      child_attrs += f' id="{value}"'
                  elif key == 'bgColor':
                      child_styles += f' background-color:{value};'
                  elif key == 'fgColor':
                      child_styles += f' color:{value};'
                  else:
                      child_styles += f' {key}:{value}px;'
              html_code += f'<{child_tag}{child_attrs} style="{child_styles}">'
              html_code += f'</{child_tag}>'
      html_code += f'</{tag}>'
    return html_code



In [ ]:
!git clone https://github.com/stanfordnlp/miniwob-plusplus-demos.git

Cloning into 'miniwob-plusplus-demos'...
remote: Enumerating objects: 18191, done.
remote: Total 18191 (delta 0), reused 0 (delta 0), pack-reused 18191
Receiving objects: 100% (18191/18191), 95.11 MiB | 39.56 MiB/s, done.
Resolving deltas: 100% (159/159), done.
Updating files: 100% (18125/18125), done.


In [ ]:
import os
import gzip
import json

path = "/content/miniwob-plusplus-demos"
dir_list = os.listdir(path)
dir_list

['2017-11-05_fourth-turk',
 '2017-10-26_third-turk',
 '.git',
 '2017-12-16_alaska',
 'clean-demos',
 'README.md',
 '2017-10-16_second-turk',
 '2017-09-29_first-turk']

In [ ]:
import gc
gc.collect()

0

In [ ]:
# We'll just create a normal dataset, and ensuring the casting of all types

data_read = []

for dir in dir_list[7:8]:
  if '-' in dir:
    #print(dir)
    sub_folders = os.listdir(path + '/' + dir)
    #print(sub_folders)
    i = 0
    for task_folder in sub_folders:
      print(f'{dir} - {i}/{len(sub_folders)}')
      i += 1
      task_files = os.listdir(path + '/' + dir + '/' + task_folder)
      for task_file in task_files:

        #print(task_file)
        try:
          with gzip.open(path + '/' + dir + '/' + task_folder + '/' + task_file, 'rb') as f:
            data = f.read()
            json_data = json.loads(data.decode('utf-8'))
            try:
              _ = json_data['taskName']
            except:
              json_data['taskName'] = task_folder
            data_read.append(json_data)
        except:
          #print('error: ' + path + '/' + dir + '/' + task_folder + '/' + task_file)
          # not zipped, so read as it is
          with open(path + '/' + dir + '/' + task_folder + '/' + task_file, 'r') as f:
            json_data = json.load(f)
            #print(json_data.keys())
            try:
              _ = json_data['taskName']
            except:
              json_data['taskName'] = task_folder
            data_read.append(json_data)
          #break


          #print(json_data)
        #break

  break

2017-09-29_first-turk - 0/25
2017-09-29_first-turk - 1/25
2017-09-29_first-turk - 2/25
2017-09-29_first-turk - 3/25
2017-09-29_first-turk - 4/25
2017-09-29_first-turk - 5/25
2017-09-29_first-turk - 6/25
2017-09-29_first-turk - 7/25
2017-09-29_first-turk - 8/25
2017-09-29_first-turk - 9/25
2017-09-29_first-turk - 10/25
2017-09-29_first-turk - 11/25
2017-09-29_first-turk - 12/25
2017-09-29_first-turk - 13/25
2017-09-29_first-turk - 14/25
2017-09-29_first-turk - 15/25
2017-09-29_first-turk - 16/25
2017-09-29_first-turk - 17/25
2017-09-29_first-turk - 18/25
2017-09-29_first-turk - 19/25
2017-09-29_first-turk - 20/25
2017-09-29_first-turk - 21/25
2017-09-29_first-turk - 22/25
2017-09-29_first-turk - 23/25
2017-09-29_first-turk - 24/25


In [ ]:
print(data_read[0])
len(data_read)

{'utterance': 'Select 12/02/2016 as the date and hit submit.', 'states': [{'time': 0, 'action': None, 'dom': {'tag': 'BODY', 'left': 0, 'top': 0, 'width': 1299, 'height': 210, 'children': [{'tag': 'FORM', 'left': 0, 'top': 0, 'width': 1299, 'height': 210, 'children': [{'tag': 'DIV', 'left': 0, 'top': 0, 'width': 160, 'height': 210, 'children': [{'tag': 'DIV', 'left': 0, 'top': 50, 'width': 160, 'height': 67, 'children': [{'tag': 'P', 'left': 2, 'top': 52, 'width': 156, 'height': 21, 'children': [{'tag': 't', 'left': 2, 'top': 58, 'width': 26.671875, 'height': 11, 'ref': -1, 'children': [], 'text': 'Date:'}, {'tag': 'INPUT_text', 'left': 28.671875, 'top': 52, 'width': 124, 'height': 21, 'children': [], 'id': 'datepicker', 'classes': 'hasDatepicker', 'ref': 200, 'bgColor': 'rgb(255, 255, 255)', 'fgColor': 'rgb(0, 0, 0)', 'value': '', 'text': ''}], 'id': '', 'classes': '', 'ref': 199, 'bgColor': 'rgba(0, 0, 0, 0)', 'fgColor': 'rgb(0, 0, 0)'}, {'tag': 'BUTTON', 'left': 27, 'top': 84, 'widt

771

In [ ]:
available_actions = ['mousedown', 'keydown', 'click', 'keypress', 'dblclick'] # allowed actions to record
discovered_actions = [] # list all eventual actions we can find


def process_state(state):
  #print(f'process_state: {state}')
  state_time = state['time']
  action = state['action']
  dom = state['dom']

  return state_time, action, dom

def process_episode(entry):
  #print(f'keys: {entry.keys()}')

  task_name = entry['taskName']
  utterance = entry['utterance']
  states = entry['states']
  reward = entry['reward']
  raw_reward = entry['rawReward']
  #print(f'state\'s keys: {states[0].keys()}, for a total of {len(states)}')
  #len(states)
  #print(task_name, utterance, reward, raw_reward)

  processed_states = []

  # Iterate over the states and process them
  for state in states:
    state_time, state_action, state_dom = process_state(state)
    if state_action is None:
      #print(f'got none state_action: {state_action}')
      continue
    elif state_action['type'] in available_actions:
      processed_states.append({
          'time': state_time,
          'action_type': state_action['type'],
          #'coords_x': state_action['x'],
          #'coords_y': state_action['y'],
          'dom': state_dom
      })
    else:
      if state_action['type'] not in discovered_actions:
        discovered_actions.append(state_action['type'])


  if len(processed_states) > 0:
    processed_episode = {
        'task_name': task_name,
        'utterance': utterance,
        #'states': states,
        'reward': reward,
        'raw_reward': float(raw_reward),
        'processed_states': processed_states
    }
    return processed_episode
  else:
    return None


#episode = process_episode(data_read[0])
#print(f'episode: {episode}')
processed_episodes = (list(map(process_episode, data_read)))

In [ ]:
print(f'found total of {len(processed_episodes)} episodes')
discovered_actions

found total of 771 episodes


['mouseup', 'keyup', 'scroll']

In [ ]:
import pandas as pd
df = pd.json_normalize(processed_episodes)
df.head()

,task_name,utterance,reward,raw_reward,processed_states
0,choose-date-medium,Select 12/02/2016 as the date and hit submit.,0.75005,1.0,"[{'time': 1534, 'action_type': 'mousedown', 'd..."
1,choose-date-medium,Select 12/01/2016 as the date and hit submit.,0.61155,1.0,"[{'time': 915, 'action_type': 'mousedown', 'do..."
2,choose-date-medium,Select 11/09/2016 as the date and hit submit.,0.69135,1.0,"[{'time': 1208, 'action_type': 'mousedown', 'd..."
3,choose-date-medium,Select 11/13/2016 as the date and hit submit.,0.69325,1.0,"[{'time': 700, 'action_type': 'mousedown', 'do..."
4,choose-date-medium,Select 11/20/2016 as the date and hit submit.,0.79735,1.0,"[{'time': 884, 'action_type': 'mousedown', 'do..."


In [ ]:
df.to_csv('/content/2017-09-29_first-turk.csv', index=False)

In [ ]:
from datasets import Dataset

# Define list of filenames
filenames = ['2017-11-05_fourth-turk.csv',
             '2017-10-26_third-turk.csv',
             '2017-12-16_alaska.csv',
             'clean-demos.csv',
             '2017-10-16_second-turk.csv',
             '2017-09-29_first-turk.csv']

# Read each CSV file and store dataframes in a list
dfs = [pd.read_csv(filename) for filename in filenames]

# Concatenate all dataframes into a single dataframe
df = pd.concat(dfs, ignore_index=True)

# Create HuggingFace dataset from pandas dataframe
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['task_name', 'utterance', 'reward', 'raw_reward', 'processed_states'],
    num_rows: 18124
})

In [ ]:
dataset.push_to_hub('LucasThil/miniwob_plusplus_v2_raw')

Pushing dataset shards to the dataset hub:   0%|          | 0/12 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

# Reload the dataset and reprocess some parts

In [ ]:
dataset = load_dataset("LucasThil/miniwob_plusplus_v2_raw")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/18124 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/LucasThil___parquet/LucasThil--miniwob_plusplus_v2_raw-5a839e363ad08d45/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

We need to process the DOM individually, and generate the proper action sequences for the different states by retrieving the target numbers, the target keydown text, and remove focus/target elements from the DOM. Even further, we should delete all DOM elements not needed

In [ ]:
import json

In [ ]:

drop_fields = ['children', 'tag', 'onmousemove', 'onclick', 'onscroll', 'ontouchmove', 'ontouchstart', 'ontouchend', 'onkeypress', 'children', 'bgColor', 'width', 'height', 'focused', 'fgColor', 'right', 'top', 'left', 'bottom', 'class', 'classes']

# Converts the html to a string and adds its ref numbers
def html2string(element, ref=None):
    html_conct = "<" + element['tag']
    if ref is not None:
        html_conct += ' ref=' + str(ref)

    # add fields
    for field in element:
        if not field in drop_fields and field != "text":
            html_conct += ' ' + field + '="' + element[field] + '"'
    html_conct += '>'

    # add text
    html_conct += element['text']


    # If the element doesn't match, recursively search its children
    if "children" in element:
        for i, child in enumerate(element["children"]):
            if ref is not None:
                ref += 1
            path = html2string(child, ref)
            html_conct += path

    html_conct += '</' + element['tag'] + '>'

    return html_conct


In [ ]:
example = dataset['train'][0]
import re

dom_fields = ['ref', 'parent', 'left', 'top', 'width', 'height', 'tag', 'text', 'value', 'id', 'classes', 'bg_color', 'gf_color', 'flags']

# function to iterate through the DOM elements
def iterate_dom(element):
  focused_element = None
  keydown_text = None

  # Look if the element has the 'focused': True
  if 'focused' in element.keys():
    #print(f'found "focused" for {element["tag"]}, with value {element["focused"]}')
    focused_element = element['ref']
    if 'value' in element.keys():
      keydown_text = element['value']
      #print(f'FOUND KEYDOWN TEXT "{keydown_text}", {element}')
    else:
      keydown_text = ''

  # Now delete all classes we don't need
  # todo

  # Iterate deeper
  new_children = []
  for child in element['children']:
    found_target, found_text, found_dom = iterate_dom(child)
    if found_target is not None:
      focused_element = found_target
    if found_text is not None:
      keydown_text = found_text
    new_children.append(found_dom)

  # Filter the dom
  new_dom = {}
  for field in element.keys():
    if field in dom_fields:
      new_dom[field] = element[field]

  new_dom['children'] = new_children

  return focused_element, keydown_text, new_dom


# Find target actions for each state
def preprocess_episodes(row):
  try:
    states = row['processed_states'].replace("'", '"')
  except:
    #print('error processing state')
    return None
  states = re.sub(r'\b(True|False)\b', lambda m: m.group(0).lower(), states)
  try:
    states_json = json.loads(states)
  except:
    print(f'error parsin json states: {states}')
    return None

  new_states = []

  for state in states_json:
    #print(f'action_type: {state["action_type"]}')
    #print(state['dom'])


    target_ref, keydown_text, new_dom = iterate_dom(state['dom'])
    #if 'key' in state['action_type']:
      #print(f'FOUND KEYDOWN ACTION: {state}')
      #print(f'action: {state["action_type"]}, target_ref: {target_ref}, keydown_text: {keydown_text}, new_dom: {new_dom}')

    state['refs'] = target_ref

    # modify action_type properly
    if 'key' in state['action_type']:
      state['action_type'] = 'keypress'
    else:
      state['action_type'] = 'click'

    if keydown_text is None:
      state['keydown_text'] = ''
    else:
      state['keydown_text'] = keydown_text

    state['dom'] = new_dom
    new_states.append(state)

  row['processed_states'] = new_states
  return row

i = 0
failed_rows = 0
cleaned_rows = []
for example in dataset['train']:
  if i % 100 == 0:
    print(f'{i} - failed_rows: {failed_rows}')
  #try:
  row = preprocess_episodes(example)
  if row is not None:
    cleaned_rows.append(row)
  else:
    failed_rows += 1
  #except:
   # failed_rows += 1
  i += 1
  if i >= 5000 and False:
    break

print(f'found {failed_rows} failed_rows, and total {len(cleaned_rows)} cleaned_rows')

Output hidden; open in https://colab.research.google.com to view.

1643